# Intro

In [3]:
import torch
import numpy as np
from tqdm.auto import tqdm
import seaborn as sns

import seaborn as sns
import jkutils.mpl_latex_template as mplt
import matplotlib.pyplot as plt

Modules to reload:
all-except-skipped

Modules to skip:



In [5]:
import seml
df_results = seml.get_results('graph_cert_sparse_smoothing', to_data_frame=True, fields=['config', 'stats'])

  0%|          | 0/481 [00:00<?, ?it/s]

  0%|          | 0/481 [00:00<?, ?it/s]

481


In [30]:
def plot_results_dict(plot_dict, xlabel, ylabel,
                      error_alpha=0.5, x_lim_factor=1.05, y_lim_factor=1.1,
                      legend_params={}, xlabel_params={}, ylabel_params={}, yticks_params=None, title_params=None,
                      linewidth=1, clip_on=False, zorder=3, budget_scaling=1, xlim=None, ylim=None):
    pal = sns.color_palette('colorblind', n_colors=len(plot_dict))

    max_budget = 0
    max_metric = 0

    for i, k in enumerate(plot_dict.keys()):
        if 'label' in plot_dict[k].keys():
            label = plot_dict[k]['label']
        else:
            label = k

        budgets = plot_dict[k]['budgets'] * budget_scaling
        metrics = plot_dict[k]['cert_metrics']
        
        plt.step(budgets, metrics, label=label, color=pal[i], zorder=zorder, clip_on=clip_on, linewidth=linewidth, where='mid')

        if 'errors' in plot_dict[k].keys():
            errors = plot_dict[k]['errors']
            plt.fill_between(budgets, metrics-errors, metrics+errors, alpha=error_alpha, color=pal[i], zorder=zorder, clip_on=clip_on, linewidth=0, step="mid")

        #print(f'Max error for {k} is {errors.max()}')

        max_budget = max(max_budget, budgets.max())
        max_metric = max(max_metric, metrics.max())

    if xlim is not None:
        plt.xlim(0, xlim)
    elif x_lim_factor is not None:
        plt.xlim(0, x_lim_factor * max_budget)
    else:
        plt.xlim(left=0)

    if ylim is not None:
        plt.ylim(0, ylim)
    elif y_lim_factor is not None:
        plt.ylim(0, y_lim_factor * max_metric)
    else:
        plt.ylim(bottom=0)

    plt.xlabel(xlabel, **xlabel_params)
    plt.ylabel(ylabel, **ylabel_params)

    if title_params is not None:
        plt.title(**title_params)

    if yticks_params is not None:
        plt.yticks(**yticks_params)

    plt.legend(**legend_params)

In [8]:
path = "anon/plots/sparse_smoothing/"

In [18]:
cond = (df_results['config.hparams.task'] == 'graph_classification') &\
        (df_results['config.hparams.p_adj_plus'] == 0) &\
        (df_results['config.hparams.p_adj_minus'] == 0) &\
        (df_results['config.hparams.p_att_plus'] > 0) &\
        (df_results['config.hparams.p_att_minus'] > 0) 
        #(df_results['config.hparams.hidden_channels'] == 64) 
print(len(df_results[cond]))

11


In [19]:
format = 'pgf'
for id,row in tqdm(df_results[cond].iterrows()):
    conf_id = row['config.overwrite']
    result = torch.load(f"anon/results/sparse_smoothing/post/graph_cert_sparse_smoothing_post_{conf_id}")
    result = result['cert_acc']
    for cert in ['binary_class_cert', 'multi_class_cert']:
        filename = f"{row['config.hparams.task']}-{row['config.hparams.dataset']}-{row['config.hparams.arch']}-hidden={row['config.hparams.hidden_channels']}-p_adj_plus={row['config.hparams.p_adj_plus']}-p_adj_minus={row['config.hparams.p_adj_minus']}-p_att_plus={row['config.hparams.p_att_plus']}-p_att_minus={row['config.hparams.p_att_minus']}-{cert}"
        #print(filename)

        pca = 1
        d = {
            1: {
                'budgets': np.array(result[(cert,pca,1)][0]),
                'cert_metrics': np.array(result[(cert,pca,1)][1]),
                'errors': np.array(result[(cert,pca,1)][2]),
            },
            2: {
                'budgets': np.array(result[(cert,pca,2)][0]),
                'cert_metrics': np.array(result[(cert,pca,2)][1]),
                'errors': np.array(result[(cert,pca,2)][2]),
            },
            4: {
                'budgets': np.array(result[(cert,pca,4)][0]),
                'cert_metrics': np.array(result[(cert,pca,4)][1]),
                'errors': np.array(result[(cert,pca,4)][2]),
            }
        }

        sns.set_theme()
        mplt.newfig(width=0.25, ratio_yx=1.15)
        plot_results_dict(d, 'Edit distance $\epsilon$', 'Cert. Acc.', legend_params={'fontsize': 7, 'title_fontsize': 9, 'title': '$c_X^-$'}, ylim=1, xlim=10, clip_on=True, title_params={'label': '$c_X^+=1$', 'fontsize': 9},
                        yticks_params={'ticks': [0, 0.25, 0.5, 0.75, 1.0], 'labels': ['0%', '25%', '50%', '75%', '100%']})

        mplt.savefig(path+filename+"-A", format=format, close_fig='True', preview='png', tight={'pad': 0.55})

        pcd = 1
        d = {
            1: {
                'budgets': np.array(result[(cert,1,pcd)][0]),
                'cert_metrics': np.array(result[(cert,1,pcd)][1]),
                'errors': np.array(result[(cert,1,pcd)][2]),
            },
            2: {
                'budgets': np.array(result[(cert,2, pcd)][0]),
                'cert_metrics': np.array(result[(cert,2, pcd)][1]),
                'errors': np.array(result[(cert,2, pcd)][2]),
            },
            4: {
                'budgets': np.array(result[(cert,4, pcd)][0]),
                'cert_metrics': np.array(result[(cert,4, pcd)][1]),
                'errors': np.array(result[(cert,4, pcd)][2]),
            }
        }
        sns.set_theme()
        mplt.newfig(width=0.25, ratio_yx=1.15)
        plot_results_dict(d, 'Edit distance $\epsilon$', 'Cert. Acc.', legend_params={'fontsize': 7, 'title_fontsize': 9, 'title': '$c_X^+$'}, ylim=1, xlim=10, clip_on=True, title_params={'label': '$c_X^-=1$', 'fontsize': 9},
                        yticks_params={'ticks': [0, 0.25, 0.5, 0.75, 1.0], 'labels': ['0%', '25%', '50%', '75%', '100%']})
        mplt.savefig(path+filename+"-B", format=format, close_fig='True', preview='png', tight={'pad': 0.55})
    #break

0it [00:00, ?it/s]

# Joint plots

In [40]:
cond = (df_results['config.hparams.task'] == 'node_classification') &\
               (df_results['config.hparams.p_adj_plus'] > 0) &\
        (df_results['config.hparams.p_adj_minus'] > 0)  &\
        (df_results['config.hparams.p_att_plus'] > 0) &\
        (df_results['config.hparams.p_att_minus'] > 0) &\
        (df_results['config.hparams.dataset'] == 'Cora') &\
        (df_results['config.hparams.hidden_channels'] == 32) 
        #(df_results['config.hparams.arch'] == 'GCN') 

        #(df_results['config.hparams.hidden_channels'] == 64) 
print(len(df_results[cond]))
df_results[cond][['config.overwrite', 
                'config.hparams.arch',
                 'config.hparams.p_att_plus',
                 'config.hparams.p_att_minus',
                 'config.hparams.p_adj_plus',
                 'config.hparams.p_adj_minus']]

5


,config.overwrite,config.hparams.arch,config.hparams.p_att_plus,config.hparams.p_att_minus,config.hparams.p_adj_plus,config.hparams.p_adj_minus
429,758,GCN,0.010,0.6,0.001,0.4
463,794,GCN,0.001,0.8,0.001,0.8
465,796,APPNP,0.001,0.8,0.001,0.8
471,810,APPNP,0.010,0.8,0.010,0.6
477,817,APPNP,0.010,0.8,0.010,0.8


In [44]:
result = torch.load("anon/results/sparse_smoothing/post/graph_cert_sparse_smoothing_post_794")['cert_acc']

In [45]:
result.keys()

dict_keys([('binary_class_cert', 1, 1, 1, 1), ('binary_class_cert', 1, 1, 1, 2), ('binary_class_cert', 1, 1, 1, 4), ('binary_class_cert', 1, 1, 2, 1), ('binary_class_cert', 1, 1, 2, 2), ('binary_class_cert', 1, 1, 2, 4), ('binary_class_cert', 1, 1, 4, 1), ('binary_class_cert', 1, 1, 4, 2), ('binary_class_cert', 1, 1, 4, 4), ('binary_class_cert', 1, 2, 1, 1), ('binary_class_cert', 1, 2, 1, 2), ('binary_class_cert', 1, 2, 1, 4), ('binary_class_cert', 1, 2, 2, 1), ('binary_class_cert', 1, 2, 2, 2), ('binary_class_cert', 1, 2, 2, 4), ('binary_class_cert', 1, 2, 4, 1), ('binary_class_cert', 1, 2, 4, 2), ('binary_class_cert', 1, 2, 4, 4), ('binary_class_cert', 1, 4, 1, 1), ('binary_class_cert', 1, 4, 1, 2), ('binary_class_cert', 1, 4, 1, 4), ('binary_class_cert', 1, 4, 2, 1), ('binary_class_cert', 1, 4, 2, 2), ('binary_class_cert', 1, 4, 2, 4), ('binary_class_cert', 1, 4, 4, 1), ('binary_class_cert', 1, 4, 4, 2), ('binary_class_cert', 1, 4, 4, 4), ('binary_class_cert', 2, 1, 1, 1), ('binary_

In [47]:
import matplotlib.pyplot as plt


for pca_A in [1,2,4]:
    for pcd_A in [1,2,4]:
        for pca_F in [1,2,4]:
            for pcd_F in [1,2,4]:
                x = result[(cert,pca_A,pcd_A, pca_F, pcd_F)][0]
                x[-1] = 10
                y = result[(cert,pca_A,pcd_A, pca_F, pcd_F)][1]
                label = f"A: ca={pca_A}, cd={pcd_A}. F: ca={pca_F}, cd={pcd_F}"
                plt.step(x, y, label=label, where='mid')
                plt.xlim([0,10])
                plt.legend(loc="upper right")
                mplt.savefig(path + "/" + label + ".pdf", format='pdf', close_fig='True', preview='png', tight={'pad': 0.55})

In [54]:
path = "anon/plots/sparse_smoothing/joint"
format = 'pgf'
cert = 'multi_class_cert'

In [59]:
d = {}
cA_p = 1
cA_m = 1
cF_p = 1
cF_m = 1
key = (cert, cA_p, cA_m, cF_p, cF_m)
d[1] = {
        'budgets': np.array(result[key][0]),
        'cert_metrics': np.array(result[key][1]),
        'errors': np.array(result[key][2]),
    }
cA_p = 1
cA_m = 2
cF_p = 1
cF_m = 2
key = (cert, cA_p, cA_m, cF_p, cF_m)
d[2] = {
        'budgets': np.array(result[key][0]),
        'cert_metrics': np.array(result[key][1]),
        'errors': np.array(result[key][2]),
    }

cA_p = 1
cA_m = 4
cF_p = 1
cF_m = 4
key = (cert, cA_p, cA_m, cF_p, cF_m)
d[4] = {
        'budgets': np.array(result[key][0]),
        'cert_metrics': np.array(result[key][1]),
        'errors': np.array(result[key][2]),
    }
sns.set_theme()
mplt.newfig(width=0.25, ratio_yx=1.15)
plot_results_dict(d, 'Edit distance $\epsilon$', 'Cert. Acc.', legend_params={'fontsize': 7, 'title_fontsize': 9, 'title': '$c_A^-=c_F^-$'}, ylim=1, xlim=10, clip_on=True, title_params={'label': '$c_A^+=c_F^+=1$', 'fontsize': 9},
                yticks_params={'ticks': [0, 0.25, 0.5, 0.75, 1.0], 'labels': ['0%', '25%', '50%', '75%', '100%']})
mplt.savefig(path+"A", format=format, close_fig='True', preview='png', tight={'pad': 0.55})

In [60]:
d = {}
cA_p = 1
cA_m = 1
cF_p = 1
cF_m = 1
key = (cert, cA_p, cA_m, cF_p, cF_m)
d[1] = {
        'budgets': np.array(result[key][0]),
        'cert_metrics': np.array(result[key][1]),
        'errors': np.array(result[key][2]),
    }
cA_p = 2
cA_m = 1
cF_p = 2
cF_m = 1
key = (cert, cA_p, cA_m, cF_p, cF_m)
d[2] = {
        'budgets': np.array(result[key][0]),
        'cert_metrics': np.array(result[key][1]),
        'errors': np.array(result[key][2]),
    }

cA_p = 4
cA_m = 1
cF_p = 4
cF_m = 1
key = (cert, cA_p, cA_m, cF_p, cF_m)
d[4] = {
        'budgets': np.array(result[key][0]),
        'cert_metrics': np.array(result[key][1]),
        'errors': np.array(result[key][2]),
    }
sns.set_theme()
mplt.newfig(width=0.25, ratio_yx=1.15)
plot_results_dict(d, 'Edit distance $\epsilon$', 'Cert. Acc.', legend_params={'fontsize': 7, 'title_fontsize': 9, 'title': '$c_A^+=c_F^+$'}, ylim=1, xlim=10, clip_on=True, title_params={'label': '$c_A^-=c_F^-=1$', 'fontsize': 9},
                yticks_params={'ticks': [0, 0.25, 0.5, 0.75, 1.0], 'labels': ['0%', '25%', '50%', '75%', '100%']})
mplt.savefig(path+"B", format=format, close_fig='True', preview='png', tight={'pad': 0.55})

In [63]:
d = {}
cA_p = 2
cA_m = 1
cF_p = 2
cF_m = 1
key = (cert, cA_p, cA_m, cF_p, cF_m)
d[1] = {
        'budgets': np.array(result[key][0]),
        'cert_metrics': np.array(result[key][1]),
        'errors': np.array(result[key][2]),
    }
cA_p = 2
cA_m = 2
cF_p = 2
cF_m = 2
key = (cert, cA_p, cA_m, cF_p, cF_m)
d[2] = {
        'budgets': np.array(result[key][0]),
        'cert_metrics': np.array(result[key][1]),
        'errors': np.array(result[key][2]),
    }

cA_p = 2
cA_m = 4
cF_p = 2
cF_m = 4
key = (cert, cA_p, cA_m, cF_p, cF_m)
d[4] = {
        'budgets': np.array(result[key][0]),
        'cert_metrics': np.array(result[key][1]),
        'errors': np.array(result[key][2]),
    }
sns.set_theme()
mplt.newfig(width=0.25, ratio_yx=1.15)
plot_results_dict(d, 'Edit distance $\epsilon$', 'Cert. Acc.', legend_params={'fontsize': 7, 'title_fontsize': 9, 'title': '$c_A^-=c_F^-$'}, ylim=1, xlim=10, clip_on=True, title_params={'label': '$c_A^+=c_F^+=2$', 'fontsize': 9},
                yticks_params={'ticks': [0, 0.25, 0.5, 0.75, 1.0], 'labels': ['0%', '25%', '50%', '75%', '100%']})
mplt.savefig(path+"C", format=format, close_fig='True', preview='png', tight={'pad': 0.55})

In [64]:
d = {}
cA_p = 4
cA_m = 1
cF_p = 4
cF_m = 1
key = (cert, cA_p, cA_m, cF_p, cF_m)
d[1] = {
        'budgets': np.array(result[key][0]),
        'cert_metrics': np.array(result[key][1]),
        'errors': np.array(result[key][2]),
    }
cA_p = 4
cA_m = 2
cF_p = 4
cF_m = 2
key = (cert, cA_p, cA_m, cF_p, cF_m)
d[2] = {
        'budgets': np.array(result[key][0]),
        'cert_metrics': np.array(result[key][1]),
        'errors': np.array(result[key][2]),
    }

cA_p = 4
cA_m = 4
cF_p = 4
cF_m = 4
key = (cert, cA_p, cA_m, cF_p, cF_m)
d[4] = {
        'budgets': np.array(result[key][0]),
        'cert_metrics': np.array(result[key][1]),
        'errors': np.array(result[key][2]),
    }
sns.set_theme()
mplt.newfig(width=0.25, ratio_yx=1.15)
plot_results_dict(d, 'Edit distance $\epsilon$', 'Cert. Acc.', legend_params={'fontsize': 7, 'title_fontsize': 9, 'title': '$c_A^-=c_F^-$'}, ylim=1, xlim=10, clip_on=True, title_params={'label': '$c_A^+=c_F^+=4$', 'fontsize': 9},
                yticks_params={'ticks': [0, 0.25, 0.5, 0.75, 1.0], 'labels': ['0%', '25%', '50%', '75%', '100%']})
mplt.savefig(path+"D", format=format, close_fig='True', preview='png', tight={'pad': 0.55})